In [68]:
import pandas as pd
import numpy as np

# Reading data files

In [69]:
# Read training data
train_labels = pd.read_excel('../mini_dataset/Training_Set/train_labels.xlsx')
# Read test data
test_labels = pd.read_csv('../mini_dataset/Test_Set/test_labels.csv')
# Read validation data
validation_labels = pd.read_csv('../mini_dataset/Validation_Set/validation_labels.csv')

In [70]:
train_labels.head()

,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,CME,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Reading images

In [71]:
import cv2

In [72]:
#read 300 training images
train_images = []
for i in range(300):
    img = cv2.imread('../mini_dataset/Training_Set/train_images/' + str(i+1) + '.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grayscale
    img = cv2.resize(img, (28, 28))           #resize to 224x224
    train_images.append(img)                    #append to list

#array of training images
train_images = np.array(train_images)           #convert to array
print(train_images.shape)


KeyboardInterrupt: 

In [ ]:
train_images.shape

(300, 28, 28)

In [ ]:
#normalize training images
train_images = train_images.astype('float32') / 255.0
#X = (X - mu) / sigma
train_images = (train_images - np.mean(train_images)) / np.std(train_images)

In [ ]:
#reshape training images to 2D
train_images = train_images.reshape(train_images.shape[0], -1)

In [ ]:
#reshape training images to 3D
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
#covert to grayscale
train_images = np.repeat(train_images, 3, -1)

In [ ]:
#RBM architecture
from sklearn.neural_network import BernoulliRBM

#RBM model
rbm = BernoulliRBM(n_components=100, learning_rate=0.01, batch_size=10, n_iter=20, verbose=1, random_state=42)

In [ ]:
#train RBM
rbm_features_train = rbm.fit_transform(train_images)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -525.10, time = 0.07s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -634.92, time = 0.07s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -743.57, time = 0.06s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -868.81, time = 0.07s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -976.84, time = 0.08s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -1095.65, time = 0.08s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -1200.67, time = 0.08s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -1307.45, time = 0.09s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -1423.16, time = 0.08s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -1534.89, time = 0.08s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -1653.94, time = 0.08s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -1761.90, time = 0.08s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -1882.76, time = 0.08s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -1996.69, time = 0.08s
[Berno

In [ ]:
train_images.shape

(300, 28, 28)

In [ ]:
print("train_images.shape: {}".format(train_images.shape))

train_images.shape: (300, 28, 28)


# RBM 

In [ ]:
import numpy as np

# Set the number of visible and hidden units in the RBM
num_visible = 28 * 28   # number of pixels in the images
num_hidden = 100

# Initialize the weight matrix with small random values
rbm_weights = np.random.randn(num_visible, num_hidden) * 0.1

# Initialize the visible bias vector with small random values
rbm_visible_bias = np.zeros((1, num_visible))

# Initialize the hidden bias vector with small random values
rbm_hidden_bias = np.zeros((1, num_hidden))

In [ ]:
# Sample only one training example from 300 training examples
train_example = train_images[0]

# Present the training example to the RBM as the visible l
#  rbm_visible shape = (1, 784)
rbm_visible = train_example.reshape(1, num_visible)

In [ ]:
hidden_probabilities = 1.0 / (1.0 + np.exp(-np.dot(rbm_visible, rbm_weights) - rbm_hidden_bias))

In [ ]:
print("rbm visible shape: ", rbm_visible.shape)
print("rbm weights shape: ", rbm_weights.shape)
print("rbm hidden bias shape: ", rbm_hidden_bias.shape)
print("hidden probabilities shape: ", hidden_probabilities.shape)
print("train example shape: ", train_example.shape)

rbm visible shape:  (1, 784)
rbm weights shape:  (784, 100)
rbm hidden bias shape:  (1, 100)
hidden probabilities shape:  (1, 100)
train example shape:  (28, 28)


In [ ]:
# Sample the hidden layer activations from the probabilities
hidden_states = hidden_probabilities > np.random.rand(num_hidden)

In [ ]:
# Reconstruct the visible layer activations from the hidden layer activations
reconstructed_visible_probabilities = 1.0 / (1.0 + np.exp(-np.dot(hidden_states, rbm_weights.T) - rbm_visible_bias))

In [ ]:
# Calculate the probabilities of the hidden layer activations given the reconstructed visible layer activations
reconstructed_hidden_probabilities = 1.0 / (1.0 + np.exp(-np.dot(reconstructed_visible_probabilities, rbm_weights) - rbm_hidden_bias))

In [ ]:
# Calculate the gradients of the weight matrix and biases
pos_grad_weights = np.dot(rbm_visible.T, hidden_probabilities)
neg_grad_weights = np.dot(reconstructed_visible_probabilities.T, reconstructed_hidden_probabilities)
pos_grad_visible_bias = np.mean(rbm_visible, axis=0)
neg_grad_visible_bias = np.mean(reconstructed_visible_probabilities, axis=0)
pos_grad_hidden_bias = np.mean(hidden_probabilities, axis=0)
neg_grad_hidden_bias = np.mean(reconstructed_hidden_probabilities, axis=0)

In [ ]:
# Update the weight matrix and biases
rbm_weights += 0.01 * (pos_grad_weights - neg_grad_weights) / train_example.shape[0] 
rbm_visible_bias += 0.01 * (pos_grad_visible_bias - neg_grad_visible_bias) / train_example.shape[0]
rbm_hidden_bias += 0.01 * (pos_grad_hidden_bias - neg_grad_hidden_bias) / train_example.shape[0]

In [ ]:


def gibbs_sampling(v, W, b_h, b_v, num_iterations):
    """
    Performs Gibbs sampling on a restricted Boltzmann machine (RBM).
    
    Parameters:
    - v: visible layer of the RBM
    - W: weight matrix of the RBM
    - b_h: bias vector of the hidden layer
    - b_v: bias vector of the visible layer
    - num_iterations: number of Gibbs sampling iterations to perform
    
    Returns:
    - h: final state of the hidden layer after Gibbs sampling
    """
    
    h = np.random.binomial(1, 0.5, size=W.shape[1])  # initialize hidden layer with random binary values
    
    for i in range(num_iterations):
        v_prob = 1.0 / (1.0 + np.exp(-np.dot(h, W.T) - b_v))  # calculate probabilities of visible layer given hidden layer
        v = np.random.binomial(1, v_prob)  # sample visible layer
        h_prob = 1.0 / (1.0 + np.exp(-np.dot(v, W) - b_h))  # calculate probabilities of hidden layer given visible layer
        h = np.random.binomial(1, h_prob)  # sample hidden layer
    
    return h






In [ ]:
# train using gibbs sampling
for i in range(100):
    # sample a training example
    train_example = train_images[i]
    train_example = train_example.reshape(1, num_visible)
    
    # perform gibbs sampling
    h = gibbs_sampling(train_example, rbm_weights, rbm_hidden_bias, rbm_visible_bias, 100)
    
    # calculate the gradients of the weight matrix and biases
    pos_grad_weights = np.dot(train_example.T, h)
    neg_grad_weights = np.dot(reconstructed_visible_probabilities.T, reconstructed_hidden_probabilities)
    pos_grad_visible_bias = np.mean(train_example, axis=0)
    neg_grad_visible_bias = np.mean(reconstructed_visible_probabilities, axis=0)
    pos_grad_hidden_bias = np.mean(h, axis=0)
    neg_grad_hidden_bias = np.mean(reconstructed_hidden_probabilities, axis=0)
    
    # update the weight matrix and biases
    rbm_weights += 0.01 * (pos_grad_weights - neg_grad_weights) / train_example.shape[0] 
    rbm_visible_bias += 0.01 * (pos_grad_visible_bias - neg_grad_visible_bias) / train_example.shape[0]
    rbm_hidden_bias += 0.01 * (pos_grad_hidden_bias - neg_grad_hidden_bias) / train_example.shape[0]



In [ ]:
#read test data
test_images = []
for i in range(100):
    img = cv2.imread('../mini_dataset/Test_Set/test_images/' + str(i+1) + '.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grayscale
    img = cv2.resize(img, (28, 28))           #resize to 224x224
    test_images.append(img)                    #append to list

#array of training images
test_images = np.array(test_images)
print(test_images.shape)

(100, 28, 28)


[[[0 0 0 ... 1 1 0]]

 [[0 0 0 ... 1 1 0]]

 [[0 0 0 ... 1 1 0]]

 ...

 [[0 0 0 ... 1 1 0]]

 [[0 0 0 ... 1 1 0]]

 [[0 0 0 ... 1 1 0]]]
